In [1]:
!pip install pytorch-tabnet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.5/44.5 kB 2.9 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from pytorch_tabnet.tab_model import TabNetRegressor
import torch


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
df = pd.read_excel('/content/drive/MyDrive/Nhóm 7/Đồ án môn học/DATA_TRAINING/final_dataset_after_quality_control_V2.xlsx')
df.head()

,mssv,namsinh,gioitinh,dtb_toankhoa,dtb_tichluy,sotc_tichluy,diemtbhk_1,diemtbhk_2,diemtbhk_3,diemtbhk_4,diemtbhk_5,diemtbhk_6,diemtbhk_7,diemtbhk_8,diemtbhk_9,diemtbhk_10,diemtbhk_11,diemtbhk_12
0,0000AC05XPvAibaEXe9B2tolTZ0JLoBGbkQixQS6,13,0,7.91,7.91,62,7.79,8.28,7.56,0.00,5.9075,5.9075,5.9075,5.9075,5.9075,5.907500,5.907500,5.907500
1,0001EB57XPvAibaEXe/twT+sf632fUXnsgPGeB4G,13,0,8.79,8.79,129,8.84,9.00,9.11,8.75,8.5400,8.7400,8.4100,8.7700,8.7700,8.770000,8.770000,8.770000
2,00046394XPvAibaEXe+fmxcqgvribEcT4YmJhSFD,10,0,7.26,7.26,147,5.41,7.07,6.36,8.32,7.8600,8.0900,6.9200,7.8000,8.2000,7.336667,7.336667,7.336667
3,0006A0BBXPvAibaEXe/lMOwHQdw54DgUkWaqwb1u,13,0,7.27,7.27,26,7.27,7.27,7.27,7.27,7.2700,7.2700,7.2700,7.2700,7.2700,7.270000,7.270000,7.270000
4,000AD0D8XPvAibaEXe+RQyZpP6sq6qqIPZXybx3Q,15,0,8.08,8.08,136,8.85,8.64,8.41,8.05,7.4600,7.2800,8.5500,7.2400,8.0600,8.060000,8.060000,8.060000


In [5]:
def calculate_metrics(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true, y_pred)
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100 if np.all(y_true != 0) else np.nan
    mpe = np.mean((y_true - y_pred) / y_true) * 100 if np.all(y_true != 0) else np.nan
    r2 = r2_score(y_true, y_pred)
    return {"R2 Score": r2, "MSE": mse, "RMSE": rmse, "MAE": mae, "MAPE": mape, "MPE": mpe}

In [6]:
models = {
    "TabNetRegressor": TabNetRegressor(
        n_d=8, n_a=8, n_steps=3,
        #random_state=42
    )
}

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


In [7]:
initial_columns = ['namsinh', 'gioitinh', 'dtb_toankhoa', 'dtb_tichluy', 'sotc_tichluy', 'diemtbhk_1']

# Lặp qua từng cột làm y
results = []
current_X_columns = initial_columns.copy()

for i in range(len(initial_columns), len(df.columns) - 1):
    target_column = df.columns[i+1]

    # Xác định X và y
    X = df[current_X_columns]
    y = df[target_column]

    # Chia dữ liệu 85-15
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)

    # Huấn luyện và đánh giá mô hình
    for model_name, model in models.items():
        model.fit(
            X_train.values, y_train.values.reshape(-1, 1),
            max_epochs=50,
            patience=10
        )

        # Dự đoán
        y_pred = model.predict(X_test.values)

        # Tính toán các độ đo
        metrics = calculate_metrics(y_test, y_pred)
        metrics["Model"] = model_name
        metrics["Target Column"] = target_column
        results.append(metrics)

    # Cập nhật X cho vòng tiếp theo
    current_X_columns.append(target_column)

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:687: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)


epoch 0  | loss: 19.85674|  0:00:01s
epoch 1  | loss: 2.7297  |  0:00:01s
epoch 2  | loss: 1.33556 |  0:00:02s
epoch 3  | loss: 1.02827 |  0:00:03s
epoch 4  | loss: 0.96074 |  0:00:04s
epoch 5  | loss: 0.91718 |  0:00:04s
epoch 6  | loss: 0.91972 |  0:00:05s
epoch 7  | loss: 0.86054 |  0:00:05s
epoch 8  | loss: 0.85674 |  0:00:06s
epoch 9  | loss: 0.84708 |  0:00:06s
epoch 10 | loss: 0.8552  |  0:00:07s
epoch 11 | loss: 0.81493 |  0:00:08s
epoch 12 | loss: 0.80564 |  0:00:08s
epoch 13 | loss: 0.81254 |  0:00:09s
epoch 14 | loss: 0.82061 |  0:00:09s
epoch 15 | loss: 0.85139 |  0:00:10s
epoch 16 | loss: 0.845   |  0:00:10s
epoch 17 | loss: 0.80515 |  0:00:11s
epoch 18 | loss: 0.77481 |  0:00:12s
epoch 19 | loss: 0.78744 |  0:00:12s
epoch 20 | loss: 0.77734 |  0:00:13s
epoch 21 | loss: 0.78587 |  0:00:14s
epoch 22 | loss: 0.80721 |  0:00:15s
epoch 23 | loss: 0.7742  |  0:00:16s
epoch 24 | loss: 0.76946 |  0:00:16s
epoch 25 | loss: 0.7834  |  0:00:17s
epoch 26 | loss: 0.791   |  0:00:18s
e

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:687: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)


epoch 0  | loss: 18.2751 |  0:00:00s
epoch 1  | loss: 1.61994 |  0:00:01s
epoch 2  | loss: 1.00744 |  0:00:01s
epoch 3  | loss: 0.85031 |  0:00:02s
epoch 4  | loss: 0.74387 |  0:00:02s
epoch 5  | loss: 0.71285 |  0:00:03s
epoch 6  | loss: 0.67199 |  0:00:04s
epoch 7  | loss: 0.67    |  0:00:04s
epoch 8  | loss: 0.64916 |  0:00:05s
epoch 9  | loss: 0.61983 |  0:00:05s
epoch 10 | loss: 0.6225  |  0:00:06s
epoch 11 | loss: 0.62437 |  0:00:07s
epoch 12 | loss: 0.62365 |  0:00:07s
epoch 13 | loss: 0.60703 |  0:00:08s
epoch 14 | loss: 0.5944  |  0:00:08s
epoch 15 | loss: 0.60142 |  0:00:09s
epoch 16 | loss: 0.59685 |  0:00:10s
epoch 17 | loss: 0.56365 |  0:00:11s
epoch 18 | loss: 0.56868 |  0:00:12s
epoch 19 | loss: 0.56769 |  0:00:13s
epoch 20 | loss: 0.5547  |  0:00:13s
epoch 21 | loss: 0.56466 |  0:00:14s
epoch 22 | loss: 0.572   |  0:00:15s
epoch 23 | loss: 0.5864  |  0:00:15s
epoch 24 | loss: 0.57896 |  0:00:16s
epoch 25 | loss: 0.55403 |  0:00:17s
epoch 26 | loss: 0.55268 |  0:00:17s
e

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:687: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)


...

epoch 0  | loss: 45.56747|  0:00:00s
epoch 1  | loss: 21.4538 |  0:00:01s
epoch 2  | loss: 3.46305 |  0:00:02s
epoch 3  | loss: 1.64978 |  0:00:03s
epoch 4  | loss: 1.11491 |  0:00:04s
epoch 5  | loss: 0.9818  |  0:00:04s
epoch 6  | loss: 0.94733 |  0:00:05s
epoch 7  | loss: 0.96352 |  0:00:05s
epoch 8  | loss: 0.91205 |  0:00:06s
epoch 9  | loss: 0.9149  |  0:00:06s
epoch 10 | loss: 0.90199 |  0:00:07s
epoch 11 | loss: 0.8765  |  0:00:08s
epoch 12 | loss: 0.86559 |  0:00:08s
epoch 13 | loss: 0.86076 |  0:00:09s
epoch 14 | loss: 0.82075 |  0:00:09s
epoch 15 | loss: 0.85368 |  0:00:10s
epoch 16 | loss: 0.87118 |  0:00:10s
epoch 17 | loss: 0.8439  |  0:00:11s
epoch 18 | loss: 0.83706 |  0:00:12s
epoch 19 | loss: 0.81673 |  0:00:12s
epoch 20 | loss: 0.83878 |  0:00:13s
epoch 21 | loss: 0.82812 |  0:00:13s
epoch 22 | loss: 0.78992 |  0:00:14s
epoch 23 | loss: 0.80721 |  0:00:15s
epoch 24 | loss: 0.78406 |  0:00:16s
epoch 25 | loss: 0.79053 |  0:00:17s
epoch 26 | loss: 0.79305 |  0:00:18s
e

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:687: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)


epoch 0  | loss: 21.37522|  0:00:00s
epoch 1  | loss: 1.78709 |  0:00:01s
epoch 2  | loss: 0.90609 |  0:00:01s
epoch 3  | loss: 0.77173 |  0:00:02s
epoch 4  | loss: 0.72027 |  0:00:02s
epoch 5  | loss: 0.65663 |  0:00:03s
epoch 6  | loss: 0.66639 |  0:00:04s
epoch 7  | loss: 0.64837 |  0:00:04s
epoch 8  | loss: 0.63776 |  0:00:05s
epoch 9  | loss: 0.60499 |  0:00:05s
epoch 10 | loss: 0.57697 |  0:00:06s
epoch 11 | loss: 0.58613 |  0:00:07s
epoch 12 | loss: 0.58642 |  0:00:07s
epoch 13 | loss: 0.57999 |  0:00:08s
epoch 14 | loss: 0.61259 |  0:00:08s
epoch 15 | loss: 0.57602 |  0:00:09s
epoch 16 | loss: 0.55607 |  0:00:10s
epoch 17 | loss: 0.54804 |  0:00:11s
epoch 18 | loss: 0.56209 |  0:00:12s
epoch 19 | loss: 0.58967 |  0:00:13s
epoch 20 | loss: 0.57106 |  0:00:13s
epoch 21 | loss: 0.57171 |  0:00:14s
epoch 22 | loss: 0.54461 |  0:00:15s
epoch 23 | loss: 0.55446 |  0:00:15s
epoch 24 | loss: 0.55914 |  0:00:16s
epoch 25 | loss: 0.60799 |  0:00:16s
epoch 26 | loss: 0.53903 |  0:00:17s
e

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:687: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)


epoch 0  | loss: 30.54808|  0:00:00s
epoch 1  | loss: 4.35248 |  0:00:01s
epoch 2  | loss: 1.07364 |  0:00:01s
epoch 3  | loss: 0.70608 |  0:00:02s
epoch 4  | loss: 0.56044 |  0:00:02s
epoch 5  | loss: 0.47764 |  0:00:03s
epoch 6  | loss: 0.44512 |  0:00:04s
epoch 7  | loss: 0.46063 |  0:00:04s
epoch 8  | loss: 0.42506 |  0:00:05s
epoch 9  | loss: 0.42885 |  0:00:05s
epoch 10 | loss: 0.38763 |  0:00:06s
epoch 11 | loss: 0.3956  |  0:00:07s
epoch 12 | loss: 0.38791 |  0:00:08s
epoch 13 | loss: 0.39248 |  0:00:09s
epoch 14 | loss: 0.37853 |  0:00:10s
epoch 15 | loss: 0.3982  |  0:00:10s
epoch 16 | loss: 0.37251 |  0:00:11s
epoch 17 | loss: 0.39458 |  0:00:11s
epoch 18 | loss: 0.37709 |  0:00:12s
epoch 19 | loss: 0.37053 |  0:00:12s
epoch 20 | loss: 0.36461 |  0:00:13s
epoch 21 | loss: 0.37673 |  0:00:14s
epoch 22 | loss: 0.38997 |  0:00:14s
epoch 23 | loss: 0.43108 |  0:00:15s
epoch 24 | loss: 0.39243 |  0:00:15s
epoch 25 | loss: 0.36027 |  0:00:16s
epoch 26 | loss: 0.36683 |  0:00:17s
e

In [8]:
# Tạo DataFrame kết quả
results_df = pd.DataFrame(results)
results_df.to_csv("tabnet_model_results.csv", index=False)

print(results_df)

    R2 Score       MSE      RMSE       MAE  MAPE  MPE            Model  \
0   0.817124  0.817604  0.904215  0.565070   NaN  NaN  TabNetRegressor   
1   0.877731  0.485482  0.696766  0.454495   NaN  NaN  TabNetRegressor   
2   0.770569  0.971974  0.985887  0.591135   NaN  NaN  TabNetRegressor   
3   0.872658  0.523361  0.723437  0.429724   NaN  NaN  TabNetRegressor   
4   0.775711  1.029721  1.014751  0.473163   NaN  NaN  TabNetRegressor   
5   0.895796  0.441091  0.664147  0.371656   NaN  NaN  TabNetRegressor   
6   0.732275  1.352634  1.163028  0.534106   NaN  NaN  TabNetRegressor   
7   0.741649  1.269838  1.126871  0.520028   NaN  NaN  TabNetRegressor   
8   0.831056  0.729241  0.853956  0.343284   NaN  NaN  TabNetRegressor   
9   0.885680  0.462525  0.680092  0.271105   NaN  NaN  TabNetRegressor   
10  0.935170  0.243468  0.493425  0.179309   NaN  NaN  TabNetRegressor   

   Target Column  
0     diemtbhk_2  
1     diemtbhk_3  
2     diemtbhk_4  
3     diemtbhk_5  
4     diemtbhk_6

In [9]:
print(results_df['R2 Score'].mean())
print(results_df['MSE'].mean())
print(results_df['RMSE'].mean())
print(results_df['MAE'].mean())

0.8304925002036622
0.7569944892903604
0.8460522807992839
0.4302795847720926
